In [19]:
import os
import requests
from anthropic import Anthropic
import gradio as gr
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from amadeus import Client, ResponseError
import json
import gradio as gr

In [20]:
!pip install amadeus


In [21]:
load_dotenv()
amnd_api = os.getenv("AMADEUS_CLIENT_ID")
amnd_secret = os.getenv("AMADEUS_CLIENT_SECRET")
print(amnd_api,amnd_secret)


LsU7soKkUrxDeftYJbZ1zCwGwG5zcOXj AOKqNXU2IEPTaRO5


In [22]:
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise RuntimeError("Missing ANTHROPIC_API_KEY in .env")

from anthropic import Anthropic
client = Anthropic(api_key=api_key)
MODEL = "claude-sonnet-4-20250514"

In [23]:
amadeus = Client(
    client_id=amnd_api,
    client_secret=amnd_secret
)

def get_price(city1, city2, date="2025-11-05"):
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=city1,
            destinationLocationCode=city2,
            departureDate=date,
            adults=1
        )

        offers = response.data

        
        markdown = "here is the 5 choice for you: \n"
        markdown += "|---------|------|----|-----------|---------|-------|\n"

        for offer in offers[:5]:  
            price = offer["price"]["total"] 
            
            itinerary = offer["itineraries"][0]["segments"][0]

            airline = itinerary["carrierCode"]
            origin = itinerary["departure"]["iataCode"]
            destination = itinerary["arrival"]["iataCode"]
            dep_time = itinerary["departure"]["at"]
            arr_time = itinerary["arrival"]["at"]
            
            duration = itinerary["duration"]

            markdown += f" Route: {origin} -->  --> {destination} , Airline:{airline}, Departure:{dep_time}, Arrival:{arr_time}, Duration:{duration}, Price:{price}\n"

        return markdown

    except ResponseError as error:
        return f"API Error: {error}"



print(get_price("LHE", "DXB"))
    

here is the 5 choice for you: 
|---------|------|----|-----------|---------|-------|
 Route: LHE -->  --> DXB , Airline:PK, Departure:2025-11-05T09:10:00, Arrival:2025-11-05T11:30:00, Duration:PT3H20M, Price:152.98
 Route: LHE -->  --> KHI , Airline:PK, Departure:2025-11-05T17:00:00, Arrival:2025-11-05T18:45:00, Duration:PT1H45M, Price:153.04
 Route: LHE -->  --> KHI , Airline:PK, Departure:2025-11-05T11:00:00, Arrival:2025-11-05T12:45:00, Duration:PT1H45M, Price:153.04
 Route: LHE -->  --> KHI , Airline:PK, Departure:2025-11-05T17:00:00, Arrival:2025-11-05T18:45:00, Duration:PT1H45M, Price:285.52
 Route: LHE -->  --> KHI , Airline:PK, Departure:2025-11-05T17:00:00, Arrival:2025-11-05T18:45:00, Duration:PT1H45M, Price:285.52



In [30]:
from amadeus import Client, ResponseError

amadeus = Client(
    client_id=amnd_api,
    client_secret=amnd_secret
)

def details(cityA, cityB, date):
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=cityA,
            destinationLocationCode=cityB,
            departureDate=date,
            adults=1
        )

        offers = response.data
        if not offers:  
            return [{
                "price": "500.00",
                "note": "Demo price shown (Amadeus sandbox doesn’t support this route/date).",
                "flights": [{
                    "direction": "outbound",
                    "airline": "XX",
                    "origin": cityA,
                    "destination": cityB,
                    "departure": f"{date}T09:00:00",
                    "arrival": f"{date}T12:00:00",
                    "duration": "PT3H"
                }]
            }]

        results = []
        for offer in offers[:5]:  
            price = offer["price"]["total"]
            offer_info = {"price": price, "flights": []}

            for idx, itinerary in enumerate(offer["itineraries"]):
                direction = "outbound" if idx == 0 else "return"
                for segment in itinerary["segments"]:
                    airline = segment["carrierCode"]
                    origin = segment["departure"]["iataCode"]
                    destination = segment["arrival"]["iataCode"]
                    dep_time = segment["departure"]["at"]
                    arr_time = segment["arrival"]["at"]
                    duration = segment["duration"]

                    offer_info["flights"].append({
                        "direction": direction,
                        "airline": airline,
                        "origin": origin,
                        "destination": destination,
                        "departure": dep_time,
                        "arrival": arr_time,
                        "duration": duration
                    })

            results.append(offer_info)

        return results

    except ResponseError as error:
        
        return [{
            "price": "N/A",
            "note": f"Amadeus API error: {str(error)}",
            "flights": []
        }]


In [31]:
real_time = {
    "name":"details",
    "description":"Provides flight offer details including outbound and return segments: price, airline, origin, destination, departure time, arrival time, and duration.",
    
    "input_schema":{
        "type":"object",
        "properties":{
            "cityA":{
                "type":"string",
                "description":"the origin city"
            },

            
            "cityB":{
                "type":"string",
                "description":"the city where customer wants to travel"
            },

            "date":{
                "type":"string",
                "description":"the date which customer wants to travel for"
            }
        },

        "required": ["cityA","cityB","date"],
        "additionalProperties": False
    }
}

tools = [real_time]

In [28]:

!pip install transformers


In [29]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu


In [3]:
!pip install diffusers transformers accelerate safetensors

import torch
from diffusers import StableDiffusionPipeline

# pick one: "runwayml/stable-diffusion-v1-5" or "stabilityai/stable-diffusion-2-1"
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe = pipe.to("cpu")   # force CPU since no GPU

prompt = "Beautiful view of Paris with Eiffel Tower, cinematic photography"

image = pipe(prompt).images[0]
image.save("paris.png")


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

text_encoder/model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


  0%|          | 0/50 [00:00<?, ?it/s]

In [26]:
def image_generation(city):
    model_id = "runwayml/stable-diffusion-v1-5"
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
    pipe = pipe.to("cpu")  
    # I use pipe.to('cpu'), because I dont have GPU
    prompt = f"Give the beautiful view of unqiue or famous icon that represent the {city}"
    image = pipe(prompt).images[0]
    

In [27]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."


In [32]:
def chat(history):
    if history is None:   
        history = []

    if not history:
        return history, None

    message = history[-1]["content"]

    anthro_messages = []
    for m in history:
        if m["role"] in ("user", "assistant"):
            anthro_messages.append({
                "role": m["role"],
                "content": [{"type": "text", "text": m["content"]}]
            })

    anthro_messages.append({"role": "user", "content": [{"type": "text", "text": message}]})

    response = client.messages.create(
        model=MODEL,
        max_tokens=500,
        system=system_message,
        tools=tools,
        messages=anthro_messages
    )

    tool_results = []
    image = None  
    for block in response.content:
        if block.type == "tool_use":
            tool_name = block.name
            tool_args = block.input
            tool_id = block.id
            
            result = details(tool_args["cityA"], tool_args["cityB"], tool_args["date"])

            image = image_generation(tool_args["cityB"])
            tool_results.append({
                "tool_use_id": tool_id,
                "content": json.dumps({
                    "origin_city": tool_args["cityA"],
                    "destination_city": tool_args["cityB"],
                    "travel_date": tool_args["date"],
                    "price": result
                })
            })

    if not tool_results:
        
        history += [{"role": "assistant", "content": response.content[0].text}]
        return history, None
    
    followup = client.messages.create(
        model=MODEL,
        max_tokens=500,
        system=system_message,
        tools=tools,
        messages=[
            *anthro_messages,
            {"role": "assistant", "content": response.content},
            {"role": "user", "content": [
                {"type": "tool_result", "tool_use_id": tr["tool_use_id"], "content": tr["content"]}
                for tr in tool_results
            ]}
        ]
    )

    final_text = "".join(b.text for b in followup.content if b.type == "text")
    history += [{"role": "assistant", "content": final_text}]
    return history, image


In [33]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        if history is None:
            history = []
        history += [{"role": "user", "content": message}]
        return "", history

    entry.submit(
        do_entry, 
        inputs=[entry, chatbot], 
        outputs=[entry, chatbot]
    ).then(
        chat, 
        inputs=[chatbot], 
        outputs=[chatbot, image_output]
    )

    clear.click(lambda: ([], None), inputs=None, outputs=[chatbot, image_output], queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]